<a href="https://colab.research.google.com/github/skkzsh/llm-notebook/blob/main/langchain_summarize_epub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# これは何？
LangChainに本 (EPub) を要約してもらいます.

# 準備

まずモジュールをインストールします.

In [ ]:
!pip install openai tiktoken langchain unstructured

# EPubファイルを用意
ココでは例として [process_book.epub](https://github.com/Shinpeim/process-book/tree/master/release) (CCライセンス) を要約してもらいます.  
Google Driveにダウンロードしてください.  

Google DriveをColabにmountします.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


EPubファイルを配置したパスを変数として定義しておきます.

In [3]:
epub = '/content/drive/MyDrive/process_book.epub'

# EPubファイルをロード
[`UnstructuredEPubLoader`](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/epub.html) で, EPubファイルをロードできます.

In [4]:
%%time
from langchain.document_loaders import UnstructuredEPubLoader

loader = UnstructuredEPubLoader(epub)
docs = loader.load_and_split()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


CPU times: user 3.23 s, sys: 335 ms, total: 3.57 s
Wall time: 6.19 s


ロードした内容は分割されています.

In [5]:
len(docs)

9

ロードした内容を出力すると, EPubファイルの文章が格納されています.  
分割した文章は, 重複部分があることも分かります (`chunk_overlap`).

In [6]:
docs

[Document(page_content='Author: Shinpeim\n\nContributors: Kazuya Takeshima, NAKAMURA Yoshitaka, Seizan Shimazaki, Tatsuya Saito, kkosuge, moznion\n\nこの文書はなんですか？\n\nこの文書は*nix系のシステムにおけるプロセスやシグナルなどについて説明することを目的に書かれました。「プロセスとかよくわかってないからちゃんと知りたいな」みたいなひとたちが想定読者です。\n\n書いているあいだは gist で管理されていたのですが、ボリュームが大きくなったので github で管理するように変えました。\n\n目次\n\n導入\n\nプロセスの生成\n\nプロセスとファイル入出力\n\nファイルディスクリプタ\n\npreforkサーバーを作ってみよう\n\nゾンビプロセスと孤児プロセス\n\nシグナルとkill\n\nプロセスグループとフォアグランドプロセス\n\nepub と pdf\n\nepub化したもの、pdf化したものが release ディレクトリに入っています。thanks to mitukiii & moznion！## 導入\n\nマルチプロセスとスケジューリング\n\nnix系のシステムは、もともと複数のユーザーが同じコンピューターリソース(CPUやメモリー)を同時に使うことを前提に作られています。そのため、*nix系のシステムでは様々な処理が同時に行われるような仕組みになっています。実際、小規模なwebサービスでは nginx と unicorn と MySQL がひとつのマシンで同時に走っているような状況は珍しくないのではないでしょうか。\n\nいまはカジュアルに「同時に複数の処理が走っている」と言ってしまいましたが、マシンが持っているCPU(脳みそ)は限られた数なのに、どうやって複数の処理を同時に行っているのでしょうか？\u3000ひとつの脳みそでどうやって複数のことを同時に考えているのでしょうか？\u3000その答えは、「本当は同時に処理は行われていなくて、OSが目にも留まらぬ早さで複数の処理を切り替えているせいであたかも同時に複数の処理が行われているように見える」です。図にするとこういう感じ。

# 要約させる
`OPENAI_API_KEY` を設定します.

In [ ]:
%env OPENAI_API_KEY=

モデルには gpt-3.5-turbo を使います.

In [8]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

[`load_summarize_chain`](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html) で要約ができます.
- `chain_type` は `map_reduce` を指定してます.  
- 処理経過も確認するために `verbose` を `True` にしておきます (以降も同様).

In [9]:
from langchain.chains.summarize import load_summarize_chain

summary_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=True,
)

要約を実行します.

In [10]:
%%time
summary = summary_chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Author: Shinpeim

Contributors: Kazuya Takeshima, NAKAMURA Yoshitaka, Seizan Shimazaki, Tatsuya Saito, kkosuge, moznion

この文書はなんですか？

この文書は*nix系のシステムにおけるプロセスやシグナルなどについて説明することを目的に書かれました。「プロセスとかよくわかってないからちゃんと知りたいな」みたいなひとたちが想定読者です。

書いているあいだは gist で管理されていたのですが、ボリュームが大きくなったので github で管理するように変えました。

目次

導入

プロセスの生成

プロセスとファイル入出力

ファイルディスクリプタ

preforkサーバーを作ってみよう

ゾンビプロセスと孤児プロセス

シグナルとkill

プロセスグループとフォアグランドプロセス

epub と pdf

epub化したもの、pdf化したものが release ディレクトリに入っています。thanks to mitukiii & moznion！## 導入

マルチプロセスとスケジューリング

nix系のシステムは、もともと複数のユーザーが同じコンピューターリソース(CPUやメモリー)を同時に使うことを前提に作られています。そのため、*nix系のシステムでは様々な処理が同時に行われるような仕組みになっています。実際、小規模なwebサービスでは nginx と unicorn と MySQL がひとつのマシンで同時に走っているような状況は珍しくないのではないでしょうか。

いまはカジュアルに「同時に複数の処理が走っている」と言ってしまいましたが、マシンが持っているCPU(脳みそ)は限られた数なのに、どうやって複数の処理を同時に行っているのでしょうか？　ひとつの脳みそでどうやって複数のことを同時に考えているのでしょうか？　その答えは、「本当は


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"This document explains processes and signals in *nix systems, aimed at readers who want to understand them better. It covers topics such as process generation, file I/O, file descriptors, zombie processes, and signals. The document also discusses the lifecycle of a process and how they are generated through the fork system call, creating a parent-child relationship between processes. The article also touches on the relationship between multi-core CPUs and processes, and how they can be used to increase parallel processing.

The article discusses the concept of forking in Linux, which allows for the creation of multiple processes and the use of parallelism. It also explains the process tree and the use of the exec system call to replace a process's content with a new executable. The article also touches on the idea that

要約結果を出力します.

In [11]:
summary

'The article explains processes and signals in *nix systems, covering topics such as process generation, file I/O, file descriptors, zombie processes, and signals. It discusses the concept of forking, the process tree, and the use of file input/output. The article also introduces the concept of "file descriptors" and briefly touches on the use of pipes. It explains how file descriptors are duplicated but open file descriptions are shared between parent and child processes. The article also covers redirecting output and the use of sockets in a prefork server. It discusses zombie and orphan processes in Perl and signals, including SIGHUP and SIGPIPE. The article explores the concept of process groups and how they affect the behavior of signals sent to a group. It concludes with an example of creating a script and running it in the foreground.'

しかし, 要約結果が英語で返ってきてしまいました.  
それに, 要約結果も簡潔すぎるかもしれません.

# 要約を日本語で得る
要約実行時のログを見ると, デフォルトのプロンプトが英語 (「Write a concise summary of the following:」など) のため, 要約結果も英語になってしまったようです.  
そこで, 日本語のプロンプトに上書きします.  
文字数指定は適宜調整してください.

In [12]:
from langchain import PromptTemplate

template = """以下の本の内容を3000文字以内で要約してください:

########
{text}
########

要約:"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["text"],
)

ja_summary_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=prompt_template,
    combine_prompt=prompt_template,
    verbose=True,
)

要約を実行します.

In [13]:
%%time
ja_summary = ja_summary_chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
以下の本の内容を3000文字以内で要約してください:

########
Author: Shinpeim

Contributors: Kazuya Takeshima, NAKAMURA Yoshitaka, Seizan Shimazaki, Tatsuya Saito, kkosuge, moznion

この文書はなんですか？

この文書は*nix系のシステムにおけるプロセスやシグナルなどについて説明することを目的に書かれました。「プロセスとかよくわかってないからちゃんと知りたいな」みたいなひとたちが想定読者です。

書いているあいだは gist で管理されていたのですが、ボリュームが大きくなったので github で管理するように変えました。

目次

導入

プロセスの生成

プロセスとファイル入出力

ファイルディスクリプタ

preforkサーバーを作ってみよう

ゾンビプロセスと孤児プロセス

シグナルとkill

プロセスグループとフォアグランドプロセス

epub と pdf

epub化したもの、pdf化したものが release ディレクトリに入っています。thanks to mitukiii & moznion！## 導入

マルチプロセスとスケジューリング

nix系のシステムは、もともと複数のユーザーが同じコンピューターリソース(CPUやメモリー)を同時に使うことを前提に作られています。そのため、*nix系のシステムでは様々な処理が同時に行われるような仕組みになっています。実際、小規模なwebサービスでは nginx と unicorn と MySQL がひとつのマシンで同時に走っているような状況は珍しくないのではないでしょうか。

いまはカジュアルに「同時に複数の処理が走っている」と言ってしまいましたが、マシンが持っているCPU(脳みそ)は限られた数なのに、どうやって複数の処理を同時に行っているのでしょうか？　ひとつの脳みそでどうやって複数のことを同時に考えているのでしょうか？　その答えは、「本当は同時に処理は行わ

要約結果を出力します.

In [14]:
ja_summary

'本書は、*nix系のシステムにおけるプロセスやシグナル、ファイルの扱い方、プロセス間通信、プロセスグループなどについて解説している。プロセスは、OSが処理を切り替えるときの処理の単位であり、プロセスの生成にはforkというシステムコールが使われる。プロセスには、走行中、待ち状態、ブロック中の3つの状態があり、プロセスは、生成、処理中、終了のライフサイクルを持つ。また、プロセスは、プロセスツリーという階層構造を持ち、基本的に全てのプロセスには「自分を生んだ親プロセス」が存在する。本書は、UNIXを使ったプログラミングに興味がある人にとって、非常に役立つ情報を提供している。'

要約結果を日本語で, より詳細に得ることができました.

# ライセンス
[![クリエイティブ・コモンズ・ライセンス](http://i.creativecommons.org/l/by-sa/3.0/88x31.png)](http://creativecommons.org/licenses/by-sa/3.0/deed.ja)
このノートは[クリエイティブ・コモンズ 表示 - 継承 3.0 非移植 ライセンス](http://creativecommons.org/licenses/by-sa/3.0/deed.ja)の下で公開します.